In [1]:

import os
import pandas as pd
# import train_model
# import generate_synthetic
# import train_classifier
# import test_imgs
import numpy as np
import torch
from datetime import datetime
import os
import logging
from datetime import datetime
from random import seed


import split_dataset as split
import train_model

In [2]:
path_home = '/home/pathorad3090/Documents/Hadar/SyntheticEvaluation'
path_models = path_home + "/models"
path_raw_data = path_home + "/data/mnist_images"

print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [ ]:

# Set parameters for which train_size, gen_size, synthetic/real ratio, GAN train cutoff
# Set initial dir for experiments

# Set parameters as lists of integer values
seed = 42
train_sizes = [50, 100, 400, 800, 1000, 5000, 10000]  # List of different training sizes
gen_sizes = [50]  # List of different generation sizes
synthetic_real_ratio = 0.5  # Ratio of synthetic to real data
gan_train_cutoff = 5000  # Number of GAN training iterations before switching
train_ratio = 0.8
kimg = 500
# Configure logging
timestamp = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
log_filename = (f"log_{timestamp}_seed{seed}_trainSizes{'_'.join(map(str, train_sizes))}_"
                f"genSizes{'_'.join(map(str, gen_sizes))}_kimg{kimg}.log")

logging.basicConfig(    filename=log_filename, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
# Example log messages
logging.info('Logging setup complete.')
logging.info('Parameters used: seed=%d, train_sizes=%s, gen_sizes=%s, kimg=%d', seed, train_sizes, gen_sizes, kimg)

# Log the configured parameters for verification
logger.info('log_filename: '+ log_filename)
logger.info("Training Configuration:")
logger.info(f"Train Sizes: {train_sizes}")
logger.info(f"Generation Sizes: {gen_sizes}")
# logger.info(f"Synthetic/Real Ratio: {synthetic_real_ratio}")
# logger.info(f"StyleGAN2-ADA Training Cutoff: {gan_train_cutoff}")
# logger.info(f"Train Ratio: {train_ratio}")
logger.info(f"Train Cutoff: {kimg}")
logger.info("-" * 40)  # Separator for clarity


2024-11-04 20:58:55,513 - INFO - Logging setup complete.
2024-11-04 20:58:55,514 - INFO - Parameters used: seed=42, train_sizes=[50, 100, 400, 800, 1000, 5000, 10000], gen_sizes=[30], kimg=500
2024-11-04 20:58:55,514 - INFO - log_filename: log_2024_11_04__20_58_55_seed42_trainSizes50_100_400_800_1000_5000_10000_genSizes30_kimg500.log
2024-11-04 20:58:55,515 - INFO - Training Configuration:
2024-11-04 20:58:55,515 - INFO - Train Sizes: [50, 100, 400, 800, 1000, 5000, 10000]
2024-11-04 20:58:55,516 - INFO - Generation Sizes: [30]
2024-11-04 20:58:55,516 - INFO - Train Cutoff: 500
2024-11-04 20:58:55,517 - INFO - ----------------------------------------


In [4]:
# Use preprocessing.ipynb to create a proper dataset
# Distribute files to relevant subfolders + create JSON


In [ ]:
# Define paths and parameters
full_json = path_home + '/data/dataset_full.json'
train_json = full_json

logger.info(f"Loading data from {full_json} for splitting into train/test subsets.")

# Split into train/test subsets
train_df, test_df = split.split_train_test(train_json, train_ratio, seed=seed)

train_output_file = f"{path_raw_data}/train_data.json"
test_output_file = f"{path_raw_data}/test_data.json"

logger.info(f"Saving training data to {train_output_file}.")
split.save_data(train_df, train_output_file)

logger.info(f"Saving testing data to {test_output_file}.")
split.save_data(test_df, test_output_file)

logger.info("Printing class distribution for training and testing datasets.")
split.print_class_distribution(train_df, "Train")
split.print_class_distribution(test_df, "Test")

logger.info("Data splitting and saving completed successfully.")


2024-11-04 09:27:41,423 - INFO - Loading data from /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/dataset_full.json for splitting into train/test subsets.
2024-11-04 09:27:41,424 - INFO - Splitting data from /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/dataset_full.json with train ratio: 0.8
Splitting data: 100%|██████████| 10/10 [00:00<00:00, 164.00class/s]
2024-11-04 09:27:41,552 - INFO - Split completed: 47995 training samples and 12005 testing samples.
2024-11-04 09:27:41,555 - INFO - Saving training data to /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/mnist_images/train_data.json.
2024-11-04 09:27:41,556 - INFO - Saving data to /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/mnist_images/train_data.json
2024-11-04 09:27:41,617 - INFO - Data saved successfully: /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/mnist_images/train_data.json
2024-11-04 09:27:41,619 - INFO - Saving testing data to /home/pathorad3090/Docum

In [6]:
# train MLP/CNN classifier, test for benchmark using test_imgs.py
# V

In [7]:
!export MKL_SERVICE_FORCE_INTEL=1

In [ ]:
train_json = path_raw_data + '/train_data.json'

# train_sizes = [40000]

for train_size in train_sizes:
    # Create a name for the subset and the model
    model_name = f"model_{train_size}_{datetime.now().strftime('%Y%m%d_%H-%M')}"
    path_model = os.path.join(path_models, model_name)
    path_experiments = os.path.join(path_model, 'experiments')
    path_dataset = os.path.join(path_model, 'dataset')
    path_subset_json = f"{path_model}/dataset_subset_size_{train_size}_seed_{seed}.json"
    
    logger.info(f"Creating directories for model: {model_name}")
    _, _, path_model_images = split.open_folders(model_name, path_model)
    
    logger.info(f"Generating subset data from {train_json} with size {train_size} and seed {seed}")
    subset_df = split.subset_data(train_json, train_size, seed)
    
    logger.info(f"Subset DataFrame created. Path raw data: {path_raw_data}, Path model images: {path_model_images}")
    split.copy_images_to_model_and_dataset(subset_df, path_raw_data, path_model_images)
    
    logger.info(f"Saving subset data to {path_subset_json}")
    split.save_data(subset_df, path_subset_json)

    logger.info(f"Distributing files into label directories at {path_model_images}")
    split.distribute_files_to_label_dirs(path_model_images)
    
    logger.info("Generating labels JSON for the subset.")
    split.generate_labels_json(path_model_images, path_model_images, "dataset.json")
    
    logger.info(f"Creating dataset for {model_name}...")
    train_model.create_dataset(path_home, path_model_images, path_dataset)

    logger.info(f"Training model {model_name}...")
    train_model.run_stylegan_training(path_home, path_experiments, path_dataset, snap=10, kimg=kimg)
    
    logger.info(f"Cleaning up model directories for {model_name}")
    split.delete_images_and_dataset_dirs(path_model)
    
    path_latest_pkl_file = split.get_latest_pkl_file(path_model)
    if path_latest_pkl_file:
        logger.info(f"Most recent .pkl file: {path_latest_pkl_file}")
        for gen_size in gen_sizes:
            gen_size = gen_size // 10
            path_generations = os.path.join(path_model, 'generations_'+ str(gen_size * 10))
            logger.info(f"Generating synthetic images for {model_name} with generation size {gen_size * 10}...")
            train_model.generate_stylegan_images(path_home, path_latest_pkl_file, path_generations, f"0-{gen_size}")
    else:
        logger.warning("No .pkl file found for generation.")


2024-11-04 09:27:41,882 - INFO - Creating directories for model: model_0.0K_20241104_09-27
2024-11-04 09:27:41,883 - INFO - Creating directories for model: model_0.0K_20241104_09-27 in /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27
2024-11-04 09:27:41,884 - INFO - Directories created: /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27/dataset, /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27/experiments, /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27/images
2024-11-04 09:27:41,884 - INFO - Generating subset data from /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/mnist_images/train_data.json with size 50 and seed 42
2024-11-04 09:27:41,884 - INFO - Creating subset from /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/data/mnist_images/train_data.json with size: 50
Subsetting classes: 100%|██████████| 10/10 [00:0

Creating dataset with command: python /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/stylegan2-ada-pytorch/dataset_tool.py --source /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27/images --dest /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27/dataset


/home/pathorad3090/Documents/Hadar/SyntheticEvaluation/stylegan2-ada-pytorch/dataset_tool.py:205: DeprecationWarning: BOX is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BOX instead.
  resample = { 'box': PIL.Image.BOX, 'lanczos': PIL.Image.LANCZOS }[resize_filter]
/home/pathorad3090/Documents/Hadar/SyntheticEvaluation/stylegan2-ada-pytorch/dataset_tool.py:205: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  resample = { 'box': PIL.Image.BOX, 'lanczos': PIL.Image.LANCZOS }[resize_filter]
100%|██████████| 50/50 [00:00<00:00, 5185.45it/s]
2024-11-04 09:27:42,052 - INFO - Training model model_0.0K_20241104_09-27...


Running command: python /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/stylegan2-ada-pytorch/train.py --snap 10 --cond=1 --outdir /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27/experiments --data /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27/dataset --kimg=100

Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27/dataset",
    "use_labels": true,
    "max_size": 50,
    "xflip": false,
    "resolution": 32
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
  

2024-11-04 09:48:28,830 - INFO - Cleaning up model directories for model_0.0K_20241104_09-27
2024-11-04 09:48:28,831 - INFO - Deleting images and dataset directories in /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27
2024-11-04 09:48:28,831 - INFO - Moved: /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27/images/dataset.json to /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27
2024-11-04 09:48:28,832 - INFO - Deleted: /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27/images
2024-11-04 09:48:28,833 - INFO - Deleted: /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27/dataset
2024-11-04 09:48:28,833 - INFO - Searching for the latest .pkl file in /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_0.0K_20241104_09-27
2024-11-04 09:48:28,834 - INFO - Latest .pkl file found: /home/pathora

...

Images for class 8 generated successfully in /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_10.0K_20241104_11-44/generations_/30/class_8
Running command: python /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/stylegan2-ada-pytorch/generate.py --outdir=/home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_10.0K_20241104_11-44/generations_/30/class_9 --seeds=0-3 --class=9 --network=/home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_10.0K_20241104_11-44/experiments/00000-dataset-cond-auto1-kimg100/network-snapshot-000100.pkl
Loading networks from "/home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_10.0K_20241104_11-44/experiments/00000-dataset-cond-auto1-kimg100/network-snapshot-000100.pkl"...
Generating image for seed 0 (0/4) ...
Setting up PyTorch plugin "bias_act_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... 

/home/pathorad3090/Documents/Hadar/SyntheticEvaluation/stylegan2-ada-pytorch/torch_utils/ops/bias_act.py:50: UserWarning: Failed to build CUDA kernels for bias_act. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/home/pathorad3090/miniconda3/envs/MLH/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 1673, in _run_ninja_build
    env=env)
  File "/home/pathorad3090/miniconda3/envs/MLH/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['ninja', '-v']' returned non-zero exit status 1.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/pathorad3090/Documents/Hadar/SyntheticEvaluation/stylegan2-ada-pytorch/torch_utils/ops/bias_act.py", line 48, in _init
    _plugin = custom_ops.get_plugin('bias_act_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/home/pathorad309

Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Image shape: (32, 32, 3)
Generating image for seed 1 (1/4) ...
Image shape: (32, 32, 3)
Generating image for seed 2 (2/4) ...
Image shape: (32, 32, 3)
Generating image for seed 3 (3/4) ...
Image shape: (32, 32, 3)


/home/pathorad3090/Documents/Hadar/SyntheticEvaluation/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/home/pathorad3090/miniconda3/envs/MLH/lib/python3.7/site-packages/torch/utils/cpp_extension.py", line 1673, in _run_ninja_build
    env=env)
  File "/home/pathorad3090/miniconda3/envs/MLH/lib/python3.7/subprocess.py", line 512, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['ninja', '-v']' returned non-zero exit status 1.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/pathorad3090/Documents/Hadar/SyntheticEvaluation/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/home/pathora

Images for class 9 generated successfully in /home/pathorad3090/Documents/Hadar/SyntheticEvaluation/models/model_10.0K_20241104_11-44/generations_/30/class_9


In [9]:
# show results - relevant metric: avg/class accuracy, f1, precision, recall, AUC-ROC...
# graph/tabular
